<a href="https://colab.research.google.com/github/KKM2222/My-Colab/blob/main/worldcupprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#패키지 불러오기
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
#csv파일 불러오기 및 변수 설정
matches = pd.read_csv('/content/WorldCupMatches.csv')
players = pd.read_csv('/content/WorldCupPlayers.csv')
cups = pd.read_csv('/content/WorldCups.csv')
matches = matches.dropna()
players = players.dropna()
cups = cups.dropna()
plt.figure(figsize=(12,6))
#동독,서독은 독일/소련은 러시아로 통일
def replace_name(df):
    if(df['Home Team Name'] in ['German DR', 'Germany FR']):
        df['Home Team Name'] = 'Germany'
    elif(df['Home Team Name'] == 'Soviet Union'):
        df['Home Team Name'] = 'Russia'
    
    if(df['Away Team Name'] in ['German DR', 'Germany FR']):
        df['Away Team Name'] = 'Germany'
    elif(df['Away Team Name'] == 'Soviet Union'):
        df['Away Team Name'] = 'Russia'
    return df
    
matches = matches.apply(replace_name, axis='columns')

#축구팀의 딕셔너리 생성
team_name = {}
index = 0
for idx, row in matches.iterrows():
    name = row['Home Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
    name = row['Away Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1

#필요없는 요소 제거
dropped_matches = matches.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID','Win conditions',
             'Home Team Initials', 'Away Team Initials', 'Half-time Home Goals', 'Half-time Away Goals',
             'Attendance', 'City', 'MatchID', 'Stage'], 1)

#우승한 횟수를 카운팅하기
championships = cups['Winner'].map(lambda p: 'Germany' if p=='Germany FR' else p).value_counts()

#append 'Home Team Championships' and 'Away Team Championships': 월드컵 우승 횟수
dropped_matches['Home Team Championship'] = 0
dropped_matches['Away Team Championship'] = 0

def count_championship(df):
  if(championships.get(df['Home Team Name']) != None):
    df['Home Team Championship'] = championships.get(df['Home Team Name'])
  if(championships.get(df['Away Team Name']) != None):
    df['Away Team Championship'] = championships.get(df['Away Team Name'])
  return df

dropped_matches = dropped_matches.apply(count_championship, axis='columns')

#누가 이겼는지 찾기: Home win: 1, Away win: 2, Draw: 0
dropped_matches['Winner'] = '-'

def find_winner(df):
    if(int(df['Home Team Goals']) == int(df['Away Team Goals'])):
        df['Winner'] = 0
    elif(int(df['Home Team Goals']) > int(df['Away Team Goals'])):
        df['Winner'] = 1
    else:
        df['Winner'] = 2
    return df

dropped_matches = dropped_matches.apply(find_winner, axis='columns')

#id로 팀이름을 변경
def replace_team_name_by_id(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    #df['Winner'] = team_name[df['Winner']]
    return df

teamid_matches = dropped_matches.apply(replace_team_name_by_id, axis='columns')


#필요없는 요소 제거
teamid_matches = teamid_matches.drop(['Year', 'Home Team Goals', 'Away Team Goals'], 1)


#######트레이닝 단계##########

#numpy 만들기
X = teamid_matches.loc[:,['Home Team Name', 'Away Team Name', 'Home Team Championship','Away Team Championship']]
X = np.array(X).astype('float64')

#append 데이터:'home team name', 'away team name', 'home team championship', 'away team championship',을 간단하게 바꾸고 결과를 대체하기
_X = X.copy()

_X[:,0] = X[:,1]
_X[:,1] = X[:,0]
_X[:,2] = X[:,3]
_X[:,3] = X[:,2]

y = dropped_matches.loc[:,['Winner']]
y = np.array(y).astype('int')
y = np.reshape(y,(1,850))
y = y[0]


_y = y.copy()

for i in range(len(_y)):
  if(_y[i]==1):
    _y[i] = 2
  elif(_y[i] ==2):
    _y[i] = 1
    
X = np.concatenate((X,_X), axis= 0)

y = np.concatenate((y,_y))

#테스트와 트레이닝을 shuffle하고 split하기
X,y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(y_train)
#svm을 이용한 트레이닝
param_grid = {'C': [1e3],
              'gamma': [0.0001] }
svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True)
svm_model.fit(X, y)

print("Predicting on the test set")
y_pred = svm_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=range(3)))

def prediction(team1, team2):
  id1 = team_name[team1]
  id2 = team_name[team2]
  championship1 = championships.get(team1) if championships.get(team1) != None else 0
  championship2 = championships.get(team2) if championships.get(team2) != None else 0

  x = np.array([id1, id2, championship1, championship2]).astype('float64')
  x = np.reshape(x, (1,-1))
  _y = svm_model.predict_proba(x)[0]

  text = ('Chance for '+team1+' to win '+team2+' is {}\nChance for '+team2+' to win '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(_y[1]*100,_y[2]*100,_y[0]*100)
  return _y[0], text

#A팀과 B팀 예측하기
prob1, text1 = prediction('Brazil', 'Serbia')
print(text1)

<ipython-input-17-09417eac6a02>:46: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dropped_matches = matches.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID','Win conditions',
<ipython-input-17-09417eac6a02>:91: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  teamid_matches = teamid_matches.drop(['Year', 'Home Team Goals', 'Away Team Goals'], 1)


[1 1 2 ... 1 1 1]
Predicting on the test set
              precision    recall  f1-score   support

           0       0.32      0.21      0.25        87
           1       0.62      0.66      0.64       175
           2       0.58      0.66      0.61       163

    accuracy                           0.56       425
   macro avg       0.51      0.51      0.50       425
weighted avg       0.54      0.56      0.55       425

[[ 18  31  38]
 [ 20 115  40]
 [ 18  38 107]]
Chance for Brazil to win Serbia is 60.059918732304254
Chance for Serbia to win Brazil is 19.410684461692952
Chance for Brazil and Serbia draw is 20.529396806002815


<Figure size 864x432 with 0 Axes>